In [9]:
import numpy as np
import scipy.stats as sstats
import pandas as pd 

from collections import namedtuple

def PPI_ACRenrich():
    protein_protein = pd.read_csv('protein_links_v11.0_0.9.tsv', sep=' ')
    protein_protein['protein'] = protein_protein['protein'].map(lambda x: x.lstrip('9606.'))
    protein_protein['chemical'] = protein_protein['chemical'].map(lambda x: x.lstrip('9606.'))
    
    universe = pd.read_csv('protein_chemical_links_v5.0_2.1.tsv', sep='\t')
    universe['protein'] = universe['protein'].map(lambda x: x.lstrip('9606.'))
    
    dictio = {}
    
    biomart_data = pd.read_csv("biomart.tsv", 
                  sep='\t', 
                  names=["gene", "transcript", "protein", "Entrez", "Uniprot", "name"])
    
    for i in protein_protein['chemical'].unique(): 
        dictio[i] = [protein_protein['protein'][j] for j in protein_protein[protein_protein['chemical']==i].index]
    
    #Till here the code takes 15 minutes to run, too long maybe? Does Thies have any suggestions? 
    
    #This doesn't need to be done all over again, take this dataset from the other function. 
    other_setg = pd.read_csv("test_list_genes1.0.txt")
    get_ensp = pd.merge(other_setg, biomart_data, on=["Entrez"]) 
    get_ensp = get_ensp.dropna(subset=['protein'])
    get_ensp_filtered = get_ensp.drop(["gene", "transcript", "Uniprot"], axis=1)
    print(get_ensp_filtered.head(10))
    
    list_dict_ppi = []
    for key, value in dictio.items():
        results = set_enrichment(value, get_ensp_filtered['protein'], universe['protein'])
        l_results = list(results)
        di = {"drug" : key, "oddsratio" : l_results[0], "c2stat" : l_results[1], "pvalue" : l_results[2], "table" : l_results[3], "method" : l_results[4]} 
        list_dict_ppi.append(di)
    print(sorted(list_dict_ppi, key = lambda i: i['pvalue']))
    #The whole code takes 45 minutes to run! Too long? Ask Thies tomorrow. 
    
#This step doesn't need to be written again either, we can just use the function we used earlier. 

def set_enrichment(your_set, other_set, universe, abcd_values=False):
    
    resTuple = namedtuple("setEnrichmentResult", [ 'oddsratio', 'c2statistic', 'pvalue', 'table', 'method'])

    universe  = set(universe)
    your_set  = set(your_set) & universe
    other_set = set(other_set) & universe
    
    a = your_set & other_set
    b = other_set - your_set
    c = your_set - other_set
    d = universe - (your_set | other_set)
    
    table = [ [len(a), len(b)], [len(c), len(d)]]
    if min(min(table)) <= 5:
        method = 'fisher'
        oddsratio, p = sstats.fisher_exact(table)
        chi2 = None
    else:
        method = 'chi2'
        chi2, p, dof, expected = sstats.chi2_contingency(table)
        oddsratio = 100
        if table[1][0] > 0 and table[0][1] > 0:
            oddsratio = table[0][0] * table[1][1] / (table[1][0] * table[0][1])
        else:
            oddsratio = np.inf
        #fi
    #fi
    if abcd_values:
        return resTuple(oddsratio, chi2, p, [[a,b],[c,d]], method)
    else:
        return resTuple(oddsratio, chi2, p, table, method)
    #fi
#edef

def main(): 
    PPI_ACRenrich()

main() 

   Entrez          protein  name
0    4000  ENSP00000357284  LMNA
1    4000  ENSP00000357283  LMNA
2    4000  ENSP00000357282  LMNA
3    4000  ENSP00000395597  LMNA
4    4000  ENSP00000424518  LMNA
5    4000  ENSP00000357280  LMNA
6    4000  ENSP00000426535  LMNA
7    4000  ENSP00000421821  LMNA
8    4000  ENSP00000424977  LMNA
9    4000  ENSP00000292304  LMNA
[{'drug': 'ENSP00000354558', 'oddsratio': 11.176778887655692, 'c2stat': 149.3026681131942, 'pvalue': 2.4625336871912394e-34, 'table': [[35, 954], [34, 10358]], 'method': 'chi2'}, {'drug': 'ENSP00000451828', 'oddsratio': 4.470369185116225, 'c2stat': 105.90999750843385, 'pvalue': 7.716134712345395e-25, 'table': [[60, 929], [148, 10244]], 'method': 'chi2'}, {'drug': 'ENSP00000478570', 'oddsratio': 5.718841285296982, 'c2stat': 97.34575787061175, 'pvalue': 5.821924982798354e-23, 'table': [[41, 948], [78, 10314]], 'method': 'chi2'}, {'drug': 'ENSP00000362680', 'oddsratio': 3.8209406019720342, 'c2stat': 96.29839132701912, 'pvalue': 9.88